### MAIN DOCUMENT

In [1]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

### Load Data

In [104]:
#house pricing data
home_prices = pd.read_csv("Zillow Data.csv", parse_dates=["date"])
home_prices = home_prices[home_prices["date"].dt.year.between(2010, 2024)]
home_prices["YearMonth"] = home_prices["date"].dt.to_period("M")
home_prices["Year"]= home_prices["date"].dt.year
home_prices_avg = home_prices.groupby(["state_fip_code", "YearMonth","Year"])["median_estimated_home_values"].mean().reset_index()
home_prices_avg["YearMonth"] = home_prices_avg["YearMonth"].astype(str)
home_prices_avg.to_csv("average_home_values_by_state_2010_2024.csv", index=False)
print(home_prices_avg.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8750 entries, 0 to 8749
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   state_fip_code                8750 non-null   float64
 1   YearMonth                     8750 non-null   object 
 2   Year                          8750 non-null   int32  
 3   median_estimated_home_values  8750 non-null   float64
dtypes: float64(2), int32(1), object(1)
memory usage: 239.4+ KB
None


In [105]:
print(home_prices_avg[home_prices_avg["Year"]==2020])

      state_fip_code YearMonth  Year  median_estimated_home_values
120              1.0   2020-01  2020                 141977.229373
121              1.0   2020-02  2020                 142771.948688
122              1.0   2020-03  2020                 143612.258815
123              1.0   2020-04  2020                 144584.551768
124              1.0   2020-05  2020                 145502.789513
...              ...       ...   ...                           ...
8702            56.0   2020-08  2020                 287442.089280
8703            56.0   2020-09  2020                 291148.627309
8704            56.0   2020-10  2020                 294810.527893
8705            56.0   2020-11  2020                 297780.861082
8706            56.0   2020-12  2020                 300042.146836

[600 rows x 4 columns]


In [119]:
fred = pd.read_csv("The Federal Reserve Economic Data.csv")
fred['Date'] = pd.to_datetime(fred['Date'], errors='coerce')
fred = fred[fred["Date"].dt.year.between(2010, 2024)]
fred["YearMonth"] = fred["Date"].dt.to_period("M")
fred["YearMonth"] = fred["YearMonth"].astype(str)
fred["Year"]= fred["Date"].dt.year

fred = fred[(fred['Indicator'] == 'Per Capita Income') & (fred['Cadence'] == 'Annual')]
df_fred = fred[["YearMonth", "Value", "Year", "state_fip_code"]]
print(df_fred.info())
print(df_fred.head())


/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/1158067725.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  fred = pd.read_csv("The Federal Reserve Economic Data.csv")


<class 'pandas.core.frame.DataFrame'>
Index: 43848 entries, 6231 to 2027686
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   YearMonth       43848 non-null  object 
 1   Value           43848 non-null  float64
 2   Year            43848 non-null  int32  
 3   state_fip_code  43848 non-null  float64
dtypes: float64(2), int32(1), object(1)
memory usage: 1.5+ MB
None
     YearMonth    Value  Year  state_fip_code
6231   2010-01  33849.0  2010             1.0
6232   2011-01  34887.0  2011             1.0
6233   2012-01  35564.0  2012             1.0
6234   2013-01  35713.0  2013             1.0
6235   2014-01  36729.0  2014             1.0


In [120]:
#income and rent data
acs = pd.read_csv("dp03_1yr.csv")
df_selected = acs[["Year","fips","estimate_income_and_benefits_total_households_median_household_income_(dollars)"]]
df_selected["fips"]=df_selected["fips"].astype(str)
df_selected["fips"]=df_selected["fips"].str[:2]

# Display first few rows
print(df_selected.head())



   Year fips  \
0  2013    1   
1  2013    2   
2  2013    4   
3  2013    5   
4  2013    6   

   estimate_income_and_benefits_total_households_median_household_income_(dollars)  
0                                              42849                                
1                                              72237                                
2                                              48510                                
3                                              40511                                
4                                              60190                                


/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/2002407691.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,24,90,141,142) have mixed types. Specify dtype option on import or set low_memory=False.
  acs = pd.read_csv("dp03_1yr.csv")
/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/2002407691.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["fips"]=df_selected["fips"].astype(str)
/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/2002407691.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [87]:
# Group by FIPS and aggregate relevant columns
df_grouped = df_selected.groupby(["Year","fips"]).agg({
    "estimate_income_and_benefits_total_households_median_household_income_(dollars)": "mean"  # Compute sum of income per FIPS
}).reset_index()

# Display results
print(df_grouped.head())

   Year fips  \
0  2013    1   
1  2013   10   
2  2013   11   
3  2013   12   
4  2013   13   

   estimate_income_and_benefits_total_households_median_household_income_(dollars)  
0                                       42849.000000                                
1                                       46676.666667                                
2                                       51517.555556                                
3                                       46566.000000                                
4                                       49484.702703                                


In [88]:
#inbound migration

demographic = pd.read_csv("dp02_1yr.csv")

df_filter = demographic[["Year","fips","estimate_residence_1_year_ago_population_1_year_and_over_different_house_in_the_u.s._different_county_different_state"]]

df_filter["fips"]=df_filter["fips"].astype(str)
df_filter["fips"]=df_filter["fips"].str[:2]


print(df_filter[df_filter.isna().any(axis=1)])

      Year fips  \
51    2013   72   
869   2013   72   
870   2013   72   
871   2013   72   
872   2013   72   
...    ...  ...   
8890  2023   72   
8891  2023   72   
8892  2023   72   
8893  2023   72   
8894  2023   72   

     estimate_residence_1_year_ago_population_1_year_and_over_different_house_in_the_u.s._different_county_different_state  
51                                                  NaN                                                                     
869                                                 NaN                                                                     
870                                                 NaN                                                                     
871                                                 NaN                                                                     
872                                                 NaN                                                                     
...                  

/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/1501943105.py:3: DtypeWarning: Columns (3,5,7,9,15,16,17,76,77,78,79,80,81,82,83,91,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  demographic = pd.read_csv("dp02_1yr.csv")
/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/1501943105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter["fips"]=df_filter["fips"].astype(str)
/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/1501943105.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [93]:
# Group by FIPS and aggregate relevant columns
df_group = df_filter.groupby(["Year","fips"]).agg({
    "estimate_residence_1_year_ago_population_1_year_and_over_different_house_in_the_u.s._different_county_different_state": "first"  
}).reset_index()

# Display results
print(df_group.head())

   Year fips  \
0  2013    1   
1  2013   10   
2  2013   11   
3  2013   12   
4  2013   13   

  estimate_residence_1_year_ago_population_1_year_and_over_different_house_in_the_u.s._different_county_different_state  
0                                           104102.0                                                                     
1                                            36790.0                                                                     
2                                            46571.0                                                                     
3                                           529406.0                                                                     
4                                           278972.0                                                                     


In [ ]:
google_trends = pd.read_csv("google_trends.csv")

# Inspect data
print(google_trends.head())

# Rename for clarity
google_trends.rename(columns={'buy_a_house_fast': 'buy_fast', 'foreclosure_help': 'foreclosure'}, inplace=True)

In [129]:
# Rename FIPS columns to ensure consistency
home_prices_avg.rename(columns={"state_fip_code": "FIPS"}, inplace=True)
#df_grouped.rename(columns={"fips": "FIPS"}, inplace=True)
df_group.rename(columns={"fips": "FIPS"}, inplace=True)
df_fred.rename(columns={"state_fip_code": "FIPS"}, inplace=True)
df_fred.rename(columns={"Value": "Per Capita Income"}, inplace=True)

# Ensure Year is in the same format (integer)
home_prices_avg["FIPS"] = home_prices_avg["FIPS"].astype(int)
#df_grouped["FIPS"] = df_grouped["FIPS"].astype(int)
df_group["FIPS"] = df_group["FIPS"].astype(int)
df_fred["FIPS"]=df_fred["FIPS"].astype(int)

# Merge home values with income data on Year and FIPS
df_merged = pd.merge(home_prices_avg, df_fred, on=["YearMonth","Year", "FIPS"], how="inner")

# Merge the result with migration data (2013-2023)
#df_merged = pd.merge(df_merged, df_group, on=["Year", "FIPS"], how="inner")

# Display merged dataset
print(df_merged.head())
print(df_merged.tail())



   FIPS YearMonth  Year  median_estimated_home_values  Per Capita Income
0     1   2010-01  2010                 118294.358094            33849.0
1     1   2010-01  2010                 118294.358094            33686.0
2     1   2010-01  2010                 118294.358094            36797.0
3     1   2010-01  2010                 118294.358094            27743.0
4     1   2010-01  2010                 118294.358094            25072.0
       FIPS YearMonth  Year  median_estimated_home_values  Per Capita Income
42933    56   2023-01  2023                 423151.724278            64115.0
42934    56   2023-01  2023                 423151.724278           471751.0
42935    56   2023-01  2023                 423151.724278            49350.0
42936    56   2023-01  2023                 423151.724278            57474.0
42937    56   2023-01  2023                 423151.724278            54380.0


/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/2125870592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fred.rename(columns={"state_fip_code": "FIPS"}, inplace=True)
/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/2125870592.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fred.rename(columns={"Value": "Per Capita Income"}, inplace=True)
/var/folders/vk/h7pf2xqx4rz1kbr6tq5jpk6m0000gn/T/ipykernel_16061/2125870592.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

### Feature Engineering

In [135]:
#Price to income ratio and price to rent ratio (baselines)
df_merged["PIR"] = df_merged["median_estimated_home_values"] / df_merged["Per Capita Income"]

# Flag potential overvaluation
df_merged["overvalued"] = (df_merged["PIR"] > 5).astype(int)

# Check distribution
print(df_merged.info())
print(df_merged[df_merged["overvalued"]==1])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42938 entries, 0 to 42937
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   FIPS                          42938 non-null  int64  
 1   YearMonth                     42938 non-null  object 
 2   Year                          42938 non-null  int32  
 3   median_estimated_home_values  42938 non-null  float64
 4   Per Capita Income             42938 non-null  float64
 5   PIR                           42938 non-null  float64
 6   overvalued                    42938 non-null  int64  
dtypes: float64(3), int32(1), int64(2), object(1)
memory usage: 2.1+ MB
None
       FIPS YearMonth  Year  median_estimated_home_values  Per Capita Income  \
6         1   2010-01  2010                 118294.358094            23404.0   
822       1   2022-01  2022                 181416.578278            33429.0   
888       1   2023-01  2023                 

In [ ]:
#price growth vs. income growth
df["price_income_growth_diff"] = df["home_price"].pct_change() - df["income"].pct_change()

# Flag regions where home prices are rising significantly faster than incomes
df["high_growth_risk"] = (df["price_income_growth_diff"] > 0.1).astype(int)

# Check flagged cases
print(df[["ZIP_code", "price_income_growth_diff", "high_growth_risk"]].head())

In [ ]:
#search trends
# Compute a speculative sentiment score
df["speculation_score"] = df["buy_fast"] - df["foreclosure"]

# Flag speculative markets
df["speculative_market"] = (df["speculation_score"] > 0).astype(int)

print(df[["ZIP_code", "speculation_score", "speculative_market"]].head())

In [ ]:
#Normalize features
scaler = MinMaxScaler()
df[["PIR", "PRR", "price_income_growth_diff", "speculation_score"]] = scaler.fit_transform(df[["PIR", "PRR", "price_income_growth_diff", "speculation_score"]])

print(df.head())

### Data Cleaning

In [ ]:
# Identify missing values
print(df.isnull().sum())

# Fill missing values with median values (FIND WHATS BEST)
df.fillna(df.median(), inplace=True)

In [ ]:
# save data
df.to_csv("processed_housing_data.csv", index=False)
print("Processed data saved successfully!")

## Exploratory Data Analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set style
sns.set(style="whitegrid")

# Plot histograms of key features
df[["PIR", "PRR", "price_income_growth_diff", "speculation_score"]].hist(bins=30, figsize=(12, 6))
plt.suptitle("Feature Distributions")
plt.show()

In [ ]:
#correlation heatmap
plt.figure(figsize=(10,6))
sns.heatmap(df[["PIR", "PRR", "price_income_growth_diff", "speculation_score"]].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap of Key Features")
plt.show()

In [ ]:
#geographic choropleth map
import geopandas as gpd

# Load a U.S. shapefile (or use a dataset with latitude & longitude)
us_map = gpd.read_file("us_states_shapefile.shp")

# Merge with dataset
merged = us_map.merge(df, left_on="state_name", right_on="state", how="left")

# Plot affordability risk
fig, ax = plt.subplots(figsize=(12, 8))
merged.plot(column="PIR", cmap="Reds", linewidth=0.8, edgecolor="black", legend=True, ax=ax)
plt.title("Price-to-Income Ratio Across U.S. States")
plt.show()